In [1]:
# read txt document of entry points
# also iterate over all tif files (adult 3D mask images) in the same path and perform analysis 
# all analysis except column analysis 

In [2]:
#  input and save paths
data_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/'
save_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/output/stable_9_Nov/'
subtype = 'C'
start_age = 37 # not used here
stab_limit = 9 #put limit 

In [3]:
cols_names = ['0,0', '0,1', '1,1', '1,0', 
              '0,-1', '-1,-1', '-1,0', 
              '0,2','0,-2',
              '1,2', '1,3', '1,4',
              '1,-1', '1,-2', '1,-3', '1,-4',
              '2,0', '2,1', '2,2', '2,3', '2,4', 
              '2,-1', '2,-2', '2,-3', '2,-4',
              '-1,1', '-1,2', '-1,3', '-1,4',
              '-1,-2', '-1,-3', '-1,-4',
              '-2,0', '-2,1', '-2,2', '-2,3', '-2,4', 
              '-2,-1', '-2,-2', '-2,-3', '-2,-4']

cols_px_vals = {col:i+1 for i, col in enumerate(cols_names)}

In [4]:
# cols_px_vals = {'0,0': 1, '0,1': 2, '1,1': 3, '1,0': 4, 
#                 '0,-1': 5, '-1,-1': 6, '-1,0': 7, 
#                 '0,2': 8,'0,-2': 9,
#                 '1,2': 10, '1,-1':11, '1,-2': 12,
#                 '-1,2': 12,'-1,-2': 13,
#                 '-2,-1':14, '2,1':15,
#                 '-2,-2':16, '-2,-3':17}

In [5]:
import os
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm
import pandas as pd
import math
from read_roi import read_roi_zip as col_zip
from scipy import ndimage, spatial

In [6]:
# getting list of folders in data_path, where each folder has files for one neuron
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if '_P' in f]
N_folders = sorted(N_folders)
N_folders

['211104_P36_N1',
 '211104_P36_N3',
 '211107_P36_N2',
 '211113_P36_N1',
 '211114_P36_N3',
 '211121_P36.5_N3',
 '220209_P36_N1']

In [7]:
def get_cols_labeled_neuron(n_file, cols_px_vals):

    # openning the cols_zip file
    cols_path = [f for f in files_list if 'colrois' in f.lower()][0]
    Cols = col_zip(cols_path)

    # entry_point = entry_points[n_file][1:] # remove the first T element
    start_im = re.search('P(.*)_N', n_file)
    start_im = float(start_im.group(1))
    start_ana = int((start_age - start_im)/0.25)
    files_list = datautils.get_file_names(data_path+n_file, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='')
    neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
    neuron = tif.imread(neu_path)
    mask_path = [f for f in files_list if 'mask' in f.lower()][0]
    mask = tif.imread(mask_path)
    neuron[mask==0] = 0

    # in case you want to crop the image to certain timeponit-start 
    neuron = neuron[start_ana:] #remove timepoints before start_ana from the 4D image array

    if stab_limit > 1:
        lifetimes = analysis.cal_lifetimes(neuron, save=False)
        neuron = lifetimes.copy()
        neuron[neuron<stab_limit] = 0
        trans_px = lifetimes.copy()
        trans_px[trans_px>=stab_limit] = 0
        trans_px[trans_px != 0] = 1

    # get columns centroids (only three are needed but the code is simpler this way)
    cols_centroids = {}
    for col, val in Cols.items():
        if val['type'] == 'oval':
                x0 = val['left']+int(val['width']/2); a = int(val['width']/2)  # x center, half width                                       
                y0 = val['top']+int(val['height']/2); b = int(val['height']/2)  # y center, half height  
                cols_centroids[col] = (y0,x0,a,b) 
    
    #[N,2] array of centroids of all the columns
    cols_centroids_arr = np.array([(i[0],i[1]) for i in cols_centroids.values()])
    cols_names = list(cols_centroids.keys())
    # cols_px_vals = {col:i+1 for i, col in enumerate(cols_centroids.keys())}

    # get the mask of each col based on distances from centroids 
    neu_PC = np.argwhere(np.ones_like(neuron[0,0])) #getting a point cloud of all the possible px in 2D [Y,X] array
    cols_masks = np.zeros_like(neuron[0,0])
    for px in tqdm(neu_PC):
        dist, ind = spatial.KDTree(cols_centroids_arr).query(px)
        col = cols_names[ind]
        cols_masks[px[0],px[1]] = cols_px_vals[col]
    
    mask_4D = np.broadcast_to(cols_masks, neuron.shape)
    col_labeled_neuron = neuron.copy()
    col_labeled_neuron[col_labeled_neuron!=0] = 1
    col_labeled_neuron = col_labeled_neuron * mask_4D
    
    # saving the result labeled image
    col_labeled_neuron = col_labeled_neuron.astype('uint8')
    name = save_path+n_file+'_stable'+str(stab_limit)+'_s'+subtype+'_cols_labeled.tif'
    tif.imwrite(name, col_labeled_neuron, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10, 'axes':'TZYX'})

    # option for saving the mask
    name = save_path+n_file+'_stable'+str(stab_limit)+'_s'+subtype+'_cols_mask.tif'
    tif.imwrite(name, cols_masks, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10, 'axes':'YX'})

    return col_labeled_neuron, cols_masks

def calc_col_occupancy(n_file, cols_px_vals):
    col_labeled_neuron, cols_masks = get_cols_labeled_neuron(n_file, cols_px_vals)
    col_occ_df_cols = list(cols_px_vals.keys())
    col_occ_df_cols = col_occ_df_cols.insert(0,'timepoint')
    col_occ_df = pd.DataFrame(columns=col_occ_df_cols)
    for i, frame in enumerate(col_labeled_neuron):
        t = start_age+i*0.25
        result_line = {'timepoint':t}
        for col, val in cols_px_vals.items():
            result_line[col] = len(frame[frame == val])
        col_occ_df = col_occ_df.append(result_line, ignore_index=True)
    col_occ_df = col_occ_df.loc[:,col_occ_df.sum()!=0]
    col_occ_df.to_csv(save_path+n_file+'_stable'+str(stab_limit)+'_s'+subtype+'_cols_occupanies.csv')
    return col_occ_df

In [8]:
for n_file in tqdm(N_folders):
    col_occ_df = calc_col_occupancy(n_file, cols_px_vals)

  0%|          | 0/7 [00:00<?, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_CropRoi.roi']


 14%|█▍        | 1/7 [00:20<02:05, 20.94s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_colRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_CropRoi.roi']


 29%|██▊       | 2/7 [00:39<01:37, 19.58s/it]

...

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211113_P36_N1/211113_P36_N1_FlipV_R230_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211113_P36_N1/211113_P36_N1_FlipV_R230_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211113_P36_N1/211113_P36_N1_FlipV_R230_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211113_P36_N1/211113_P36_N1_FlipV_R230_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211113_P36_N1/211113_P36_N1_CropRoi.roi']


 57%|█████▋    | 4/7 [01:24<01:05, 21.88s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211114_P36_N3/211114_P36_N3_flipV_R250_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211114_P36_N3/211114_P36_N3_Z15.55_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211114_P36_N3/211114_P36_N3_FlipV_R250_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211114_P36_N3/211114_P36_N3_FlipV_R250_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211114_P36_N3/211114_P36_N3_FlipV_R250_5D.tif']


 57%|█████▋    | 4/7 [01:32<01:09, 23.03s/it]


KeyError: '0087-0152-0230'